In [ ]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
import json
from pprint import pprint
import requests
# Google Places API Key from config
from config import gkey

In [ ]:
# Import the census data into a pandas DataFrame

census_data = pd.read_csv('Census_Data.csv')
# Preview the data
mask = census_data['Population'] >= 100

census_dta = census_data[mask]
census_dta.head()

# Randomly select 700 zip codes locations that have at least 100 residents

data = census_dta.sample(n = 100)

# Visualize the DataFrame


data.head()

In [ ]:
# Create blank columns in DataFrame for lat/lng

base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
params = {
    'key' : gkey
}
data['lat'] = ""
data['lng'] = ""


# Loop through and grab the lat/lng for each of the selected zips using Google maps

for index, row in data.iterrows():
    zipcode = row['Zipcode']
    params['address'] = zipcode
    responses = requests.get(base_url, params = params).json()

    
    try:
        zip_lat = responses["results"][0]["geometry"]["location"]["lat"]
        zip_lng = responses["results"][0]["geometry"]["location"]["lng"]

        data.set_value(index, "Lat", zip_lat)
        data.set_value(index, "Lng", zip_lng)

    except (KeyError, IndexError):
        print("Error with city data. Skipping")
        continue


        
data.head()      
   



In [ ]:
# Create an empty column for bank count

base_url = "https://maps.googleapis.com/maps/api/place/radarsearch/json?"

params = {
    'keyword'  :   'bank',
    'radius'   :    8000,
    'key'      : gkey    
}

coordinates = []

data['bank_count']= " "
# Re-loop through the DataFrame and run a Google Places search to get all banks in 5 mile radius (8000 meters)

for index, row in data.iterrows() : # index here represent the actually index 
    lat = row['Lat']
    lng = row['Lng']
    
    
    params['location'] = f"{lat},{lng}"
    responses = requests.get(base_url, params = params)
    #print(responses.url)
    bank_count = len(responses.json()['results'])
    
    try:
        data.set_value(index, "bank_count", bank_count)
        
    except (KeyError, IndexError):
        print("Error with city data. Skipping")
        continue


        
data.head()   
   

In [ ]:
# Save the DataFrame as a csv
data.to_csv("zip_bank_data.csv",
                     encoding="utf-8", index=False)

In [ ]:
import matplotlib.pyplot as plt
# Build a scatter plot for each data type
plt.scatter(data["bank_count"],
            data["Population"],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Zip Codes")

# Incorporate the other graph properties
plt.title("Population vs. Bank Count by Zip Code")
plt.ylabel("Population")
plt.xlabel("Bank Count")
plt.grid(True)
plt.xlim([-2.5, 202])
plt.ylim([-2.5, 110000])

# Save the figure
#plt.savefig("output_analysis/Population_BankCount.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot for each data type
plt.scatter(data["bank_count"],
            data["Median Age"],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Zip Codes")

# Incorporate the other graph properties
plt.title("Median Age vs. Bank Count by Zip Code")
plt.ylabel("Median Age")
plt.xlabel("Bank Count")
plt.grid(True)
plt.xlim([-2.5, 202])

# Save the figure
#plt.savefig("output_analysis/Age_BankCount.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot for each data type
plt.scatter(data["bank_count"], 
            data["Per Capita Income"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="Zip Codes")

# Incorporate the other graph properties
plt.title("Per Capita Income vs. Bank Count by Zip Code")
plt.ylabel("Per Capita Income (%)")
plt.xlabel("Bank Count")
plt.grid(True)
plt.xlim([-2.5, 202])
plt.ylim([0, 165000])

# Save the figure
#plt.savefig("output_analysis/PerCapitaIncome_BankCount.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot for each data type
plt.scatter(data["bank_count"], 
            data["Poverty Rate"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="Zip Codes")

# Incorporate the other graph properties
plt.title("Poverty Rate vs. Bank Count by Zip Code")
plt.ylabel("Poverty Rate (%)")
plt.xlabel("Bank Count")
plt.grid(True)
plt.xlim([-2.5, 202])
plt.ylim([-2.5, 102])

# Save the figure
#plt.savefig("output_analysis/PovertyRate_BankCount.png")

# Show plot
plt.show()